### Описание данных в файле transactions.csv

* customer_id - идентификатор клиента
* tr_datetime - день и время совершения транзакции (дни нумеруются с начала данных)
* mcc_code - mcc-код транзакции
* tr_type - тип транзакции
* amount - сумма транзакции в условных единицах; со знаком "+" — начисление средств клиенту (приходная транзакция), "-" — списание средств (расходная транзакция)
* term_id - идентификатор терминала

### Описание задания

Цель задания выполнить последовательно все упражнения. Будет оцениваться правильность кода, и конечный результат, т.е. после прогона всех ячеек должен получится преобразованный датасет в файле features.csv.

Обратите внимание, что задания можно выполнить разными способами, конретное решение не навязывается, однако код должен быть по возможности хорошо читаемым и лаконичным.

In [ ]:
!pip install pyspark
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=33424f6efccc1c331839c92bd8019121f91d1478746d08ebeb6e8a75d3fd7f6e
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gdown
# transactions.csv сохранён на моём гугл-диске, этот модуль поможет вытягивать 
# этот файл
from pyspark.sql import types as t
from pyspark import SQLContext, Row
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [ ]:
gdown.download(
	id="1wWYtA8Y2byFdrUqQfm0n5_KQ5wZQqWuj",
	output="/content/transactions.csv",
	quiet=False
)

Downloading...
From: https://drive.google.com/uc?id=1wWYtA8Y2byFdrUqQfm0n5_KQ5wZQqWuj
To: /content/transactions.csv
100%|██████████| 47.1M/47.1M [00:01<00:00, 36.8MB/s]


'/content/transactions.csv'

In [ ]:
spark = (
    SparkSession
      .builder
      .master('local')
      .appName('de test sber')
      .getOrCreate()
)

### 1. Создать sql context

In [ ]:
sqlContext = SQLContext(spark)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Как и написано в выводе, SQLContext устарел и ему на замену пришёл SparkSession, который объединяет в себе весь функционал SparkContext и SQLContext.

### 2. Создать DataFrame из файла transactions.csv

In [ ]:
df = (
    spark
      .read.option("header", True)
      .csv('/content/transactions.csv')
)

### 3. Напечатать схему

In [ ]:
df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- tr_datetime: string (nullable = true)
 |-- mcc_code: string (nullable = true)
 |-- tr_type: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- term_id: string (nullable = true)



### 4. Отобразить первые 20 строк DataFrame-а

In [ ]:
df.show()

+-----------+------------+--------+-------+----------+-------+
|customer_id| tr_datetime|mcc_code|tr_type|    amount|term_id|
+-----------+------------+--------+-------+----------+-------+
|   79780256| 37 13:36:14|    4814|   1030|  -3144.28|   null|
|   79780256| 39 10:16:49|    4814|   1030|  -5614.79|   null|
|   79780256| 44 09:41:33|    6011|   2010|-112295.79|   null|
|   79780256| 44 09:42:44|    6011|   2010| -67377.47|   null|
|   79780256| 51 08:53:56|    4814|   1030|  -1122.96|   null|
|   79780256| 51 08:55:09|    4814|   1030|  -2245.92|   null|
|   79780256| 58 11:18:31|    6011|   2010| -67377.47|   null|
|   79780256| 59 12:29:60|    6011|   2010| -22459.16|   null|
|   79780256| 62 15:44:60|    4814|   1030|  -3368.87|   null|
|   79780256| 62 15:46:24|    4814|   1030|  -2245.92|   null|
|   79780256| 65 06:20:50|    6011|   2010| -44918.32|   null|
|   79780256| 71 11:18:04|    6011|   2010| -89836.63|   null|
|   79780256| 78 10:38:15|    6011|   2010| -78607.05| 

### 5. Посчитать количество уникальных customer_id

In [ ]:
cust_num = (
    df
      .select('customer_id').distinct()
      .count()
)

cust_num

2000

### 6. Посчитать количество уникальных term_id

In [ ]:
df.select('term_id').distinct().count()

110872

### 7. Посчитать среднее количество транзакций на одного customer_id

In [ ]:
(
    df.select(
          (f.count('amount')/cust_num)
            .alias('average_trans_count')
      ).show()
)

+-------------------+
|average_trans_count|
+-------------------+
|            514.094|
+-------------------+



### 8. Посчитать среднюю сумму транзакций на одного customer_id

In [ ]:
(
    df.select(
          (f.sum('amount')/cust_num)
            .alias('average_trans_amount')
      ).show()
)

+--------------------+
|average_trans_amount|
+--------------------+
|-1.07253172619370...|
+--------------------+



### 9. Удалить столбец term_id

In [ ]:
df = df.drop('term_id')
df.show()

+-----------+------------+--------+-------+----------+
|customer_id| tr_datetime|mcc_code|tr_type|    amount|
+-----------+------------+--------+-------+----------+
|   79780256| 37 13:36:14|    4814|   1030|  -3144.28|
|   79780256| 39 10:16:49|    4814|   1030|  -5614.79|
|   79780256| 44 09:41:33|    6011|   2010|-112295.79|
|   79780256| 44 09:42:44|    6011|   2010| -67377.47|
|   79780256| 51 08:53:56|    4814|   1030|  -1122.96|
|   79780256| 51 08:55:09|    4814|   1030|  -2245.92|
|   79780256| 58 11:18:31|    6011|   2010| -67377.47|
|   79780256| 59 12:29:60|    6011|   2010| -22459.16|
|   79780256| 62 15:44:60|    4814|   1030|  -3368.87|
|   79780256| 62 15:46:24|    4814|   1030|  -2245.92|
|   79780256| 65 06:20:50|    6011|   2010| -44918.32|
|   79780256| 71 11:18:04|    6011|   2010| -89836.63|
|   79780256| 78 10:38:15|    6011|   2010| -78607.05|
|   79780256| 81 12:27:22|    6011|   2010|-303198.63|
|   79780256| 89 02:34:24|    6011|   2010| -67377.47|
|   797802

### 10. Добавить столбец direction, который указывает "направление" транзакции, если в поле amount отрицательное значение то туда записать D, если положительное - C

In [ ]:
df = df.withColumn('direction', f.when(f.col('amount') < 0, 'D').otherwise('C'))

df.show()

+-----------+------------+--------+-------+----------+---------+
|customer_id| tr_datetime|mcc_code|tr_type|    amount|direction|
+-----------+------------+--------+-------+----------+---------+
|   79780256| 37 13:36:14|    4814|   1030|  -3144.28|        D|
|   79780256| 39 10:16:49|    4814|   1030|  -5614.79|        D|
|   79780256| 44 09:41:33|    6011|   2010|-112295.79|        D|
|   79780256| 44 09:42:44|    6011|   2010| -67377.47|        D|
|   79780256| 51 08:53:56|    4814|   1030|  -1122.96|        D|
|   79780256| 51 08:55:09|    4814|   1030|  -2245.92|        D|
|   79780256| 58 11:18:31|    6011|   2010| -67377.47|        D|
|   79780256| 59 12:29:60|    6011|   2010| -22459.16|        D|
|   79780256| 62 15:44:60|    4814|   1030|  -3368.87|        D|
|   79780256| 62 15:46:24|    4814|   1030|  -2245.92|        D|
|   79780256| 65 06:20:50|    6011|   2010| -44918.32|        D|
|   79780256| 71 11:18:04|    6011|   2010| -89836.63|        D|
|   79780256| 78 10:38:15

### 11. Столбец amount преобразовать в абсолютное значение

In [ ]:
df = df.withColumn('amount', f.abs('amount'))

df.show()

+-----------+------------+--------+-------+---------+---------+
|customer_id| tr_datetime|mcc_code|tr_type|   amount|direction|
+-----------+------------+--------+-------+---------+---------+
|   79780256| 37 13:36:14|    4814|   1030|  3144.28|        D|
|   79780256| 39 10:16:49|    4814|   1030|  5614.79|        D|
|   79780256| 44 09:41:33|    6011|   2010|112295.79|        D|
|   79780256| 44 09:42:44|    6011|   2010| 67377.47|        D|
|   79780256| 51 08:53:56|    4814|   1030|  1122.96|        D|
|   79780256| 51 08:55:09|    4814|   1030|  2245.92|        D|
|   79780256| 58 11:18:31|    6011|   2010| 67377.47|        D|
|   79780256| 59 12:29:60|    6011|   2010| 22459.16|        D|
|   79780256| 62 15:44:60|    4814|   1030|  3368.87|        D|
|   79780256| 62 15:46:24|    4814|   1030|  2245.92|        D|
|   79780256| 65 06:20:50|    6011|   2010| 44918.32|        D|
|   79780256| 71 11:18:04|    6011|   2010| 89836.63|        D|
|   79780256| 78 10:38:15|    6011|   20

### 12. Посчитать среднюю сумму транзакций на одного customer_id отдельно по каждому направлению

In [ ]:
df.groupBy('direction').agg(f.sum('amount')/cust_num).show()

+---------+--------------------+
|direction|(sum(amount) / 2000)|
+---------+--------------------+
|        D|2.6557672116860546E7|
|        C| 1.583235486741428E7|
+---------+--------------------+



### 13. Сравнить mcc коды по направлениям AND и XOR

_немного поясню, т.к. были вопросы от кандидатов_

Собрать в 2 множества все коды из транзакций с направлением D (1-е множество) и C (2-е множество). Посчитать количество пересечений кодов в обоих множествах (AND) и количество уникальных кодов в обоих множествах (XOR)

In [ ]:
# Изначально я хотел сделать следующий код:
# D_dir.intersect(C_dir).count()
# D_dir.except(C_dir).count()
# Но google colab подумал, что я использую конструкцию try-except и не давал нужного результата
# Поэтому я решил прибегнуть к временным представлениям

(
    df.select('mcc_code')
      .where(f.col('direction') == 'D')
      .createOrReplaceTempView('D_dir')
)

(
    df.select('mcc_code')
      .where(f.col('direction') == 'C')
      .createOrReplaceTempView('C_dir')
)

In [ ]:
intersect = spark.sql('''
  select * from D_dir
    intersect
  select * from C_dir
''').count()

excpt = spark.sql('''
  select * from D_dir
    except
  select * from C_dir
''').count()

In [ ]:
print(f'Количество общих значений (AND): {intersect}')

Количество общих значений (AND): 101


In [ ]:
print(f'Количество отличающихся значений (XOR): {excpt}')

Количество отличающихся значений (XOR): 82


### 14. Сравнить типы транзакций по направлениям AND и XOR

In [ ]:
# Из тех же соображений поступаю таким же образом tr_type
(
    df.select('tr_type')
      .where(f.col('direction') == 'D')
      .createOrReplaceTempView('D_dir')
)

(
    df.select('tr_type')
      .where(f.col('direction') == 'C')
      .createOrReplaceTempView('C_dir')
)

In [ ]:
intersect = spark.sql('''
  select * from D_dir
    intersect
  select * from C_dir
''').count()

excpt = spark.sql('''
  select * from D_dir
    except
  select * from C_dir
''').count()

In [ ]:
print(f'Количество общих значений (AND): {intersect}')

Количество общих значений (AND): 22


In [ ]:
print(f'Количество отличающихся значений (XOR): {excpt}')

Количество отличающихся значений (XOR): 29


In [ ]:
# Удалим эти временные представления

spark.catalog.dropTempView('D_dir')
spark.catalog.dropTempView('C_dir')
spark.catalog.listTables()

[]

### 15. Сделать pivot, в котором строки это customer_id, столбцы mcc-коды, в ячейках суммы по amount

In [ ]:
trans_piv_mcc = df.groupBy("customer_id").pivot("mcc_code").sum("amount")

trans_piv_mcc.show()

+-----------+----+----+----+----+----+----+----+--------+----------+----+----+----+----+----+--------+----+--------+--------+-----------------+------------------+---------+--------------------+----+--------+---------+------------------+----+----+----+-------+----+----+----+----+----+----+-------+----+----+----+----+----+-------+----+------------------+------------------+----+------------------+---------+----+----+----+------------------+------------------+------------------+------------------+--------+----+----+----+------------------+---------+----+------------------+------------------+----+----+----+---------+-----------------+----+--------+---------+---------+------------------+---------+------------------+----+---------+---------+----+----+--------+--------------------+------------------+----+-----------------+-------+----+-----------------+---------+------------------+----+------------------+------------------+----+----+--------+------------------+--------+---------+--------------

### 16. Сделать pivot, в котором строки это customer_id, столбцы mcc-коды, в ячейках средние и стандартные отклонения по amount 
т.е. на каждый mcc_code должно быть до 2-х столбцов со средним и стандартным отклонением

In [ ]:
trans_piv_mcc_2 = (
    df
      .groupBy("customer_id")
      .pivot("mcc_code")
      .agg(
          f.mean('amount').alias('mcc_avg'), 
          f.stddev('amount').alias('mcc_std')
      )
)

trans_piv_mcc_2.show()

+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+----------------+-----------------+------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+-----------------+------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+------------+------------+-----------------+-----------------+------------+-----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+---

### 17. Сделать pivot, в котором строки это customer_id, столбцы типы транзакций, в ячейках средние и стандартные отклонения по amount, значения должны быть разделены по направлениям
т.е. на каждый tr_type должно быть до 4-х столбцов со средним и стандартным отклонением по каждому направлению

**Подсказка:** Можно сделать расчеты отдельно для каждого направления платежей, потом присоединить к заранее подготовленному списку уникальных customer_id. Так будет проще, наглядней и меньше вероятность сделать ошибку.



---

Следуя подсказке, сделаем 3 датафрейма:

1. ДФ, содержащий уникальный клиентов;
2. ДФ, содержащий все данные по направлению D;
3. ДФ, содержащий все данные по направлению C.



In [ ]:
dist_custs = df.select('customer_id').distinct()
d_dir = df.where(f.col('direction') == 'D')
c_dir = df.where(f.col('direction') == 'C')

Сделаем pivot по полю tr_type для каждого направления и посчитаем среднее значение и стандартное отклонение

In [ ]:
d_dir = (
    d_dir
      .groupBy('customer_id')
      .pivot('tr_type')
      .agg(
          f.mean('amount').alias('D_tr_type_avg'), 
          f.stddev('amount').alias('D_tr_type_std')
      )
)

c_dir = (
    c_dir
      .groupBy('customer_id')
      .pivot('tr_type')
      .agg(
          f.mean('amount').alias('C_tr_type_avg'), 
          f.stddev('amount').alias('C_tr_type_std')
      )
)

Левым джоином присоединим 2 и 3 датафреймы к датафрейму уникальных клиентов

In [ ]:
trans_piv_type = (
    dist_custs
      .join(
          d_dir, 
          'customer_id', 
          'left'
      )
      .join(
          c_dir, 
          'customer_id', 
          'left'
      )
)

trans_piv_type.show()

+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------

### 18. Извлечь часы из столбца tr_datetime и удалить столбец tr_datetime

In [ ]:
trans_data = (
    df
      .withColumn(
          'hour', 
          f.hour(f.to_timestamp(f.split('tr_datetime', ' ')[1]))
      )
)


trans_data.show()

+-----------+------------+--------+-------+---------+---------+----+
|customer_id| tr_datetime|mcc_code|tr_type|   amount|direction|hour|
+-----------+------------+--------+-------+---------+---------+----+
|   79780256| 37 13:36:14|    4814|   1030|  3144.28|        D|  13|
|   79780256| 39 10:16:49|    4814|   1030|  5614.79|        D|  10|
|   79780256| 44 09:41:33|    6011|   2010|112295.79|        D|   9|
|   79780256| 44 09:42:44|    6011|   2010| 67377.47|        D|   9|
|   79780256| 51 08:53:56|    4814|   1030|  1122.96|        D|   8|
|   79780256| 51 08:55:09|    4814|   1030|  2245.92|        D|   8|
|   79780256| 58 11:18:31|    6011|   2010| 67377.47|        D|  11|
|   79780256| 59 12:29:60|    6011|   2010| 22459.16|        D|null|
|   79780256| 62 15:44:60|    4814|   1030|  3368.87|        D|null|
|   79780256| 62 15:46:24|    4814|   1030|  2245.92|        D|  15|
|   79780256| 65 06:20:50|    6011|   2010| 44918.32|        D|   6|
|   79780256| 71 11:18:04|    6011

Некоторые значения hour из поля tr_datetime превратились в null. Проверим, из-за чего это произошло.

In [ ]:
trans_data.where(f.col('hour').isNull()).show()

+-----------+------------+--------+-------+---------+---------+----+
|customer_id| tr_datetime|mcc_code|tr_type|   amount|direction|hour|
+-----------+------------+--------+-------+---------+---------+----+
|   79780256| 59 12:29:60|    6011|   2010| 22459.16|        D|null|
|   79780256| 62 15:44:60|    4814|   1030|  3368.87|        D|null|
|   79780256|135 06:24:60|    4814|   1030| 22459.16|        D|null|
|   58227469|381 11:52:60|    4829|   2370|150925.54|        D|null|
|   58227469|391 17:52:60|    5411|   1110|  5913.95|        D|null|
|   58227469|403 13:01:60|    5411|   1010| 15063.36|        D|null|
|   58227469|440 08:03:60|    5411|   1110|  7523.82|        D|null|
|   58227469|453 13:58:60|    4814|   1030|  3368.87|        D|null|
|   83340880|165 17:45:60|    4814|   1030| 11229.58|        D|null|
|   83340880|180 19:15:60|    6011|   2010| 44918.32|        D|null|
|   83340880|277 12:37:60|    5411|   1110| 11470.57|        D|null|
|   83340880|292 09:15:60|    4829

В результате анализа стало ясно, что значения null появились из-за того, что некоторые значения просто не смогли кастануться к типу timestamp. Случилось это из-за того, что в поле tr_datetime есть значения секунд, равные 60, хотя должны быть в интервале от 0 до 59.

Проверим, есть ли подобные проблемы на границах часов и минут. Для этого распарсим tr_datetime на 3 поля: часы, минуты и секунды (дни отбросим за ненадобностью).

In [ ]:
data_preprocess = df.withColumn('tr_datetime', f.split('tr_datetime', ' ')[1])

data_preprocess = (
    data_preprocess
      .withColumn('hour', f.split('tr_datetime', ':')[0])
      .withColumn('minute', f.split('tr_datetime', ':')[1])
      .withColumn('second', f.split('tr_datetime', ':')[2])
)

h = data_preprocess.where(f.col('hour') == '24').count()
# Исходя из значений таймштампа (присутствуют значения часа > 12), часы заданы для 
# 24 часового формата времени
m = data_preprocess.where(f.col('minute') == '60').count()


print(f'Количество проблемных строк для значений часов: {h}, минут: {m}')

Количество проблемных строк для значений часов: 0, минут: 0


Проблемных строк не оказалось. Но осталось проверить ещё две граничные ситуации: 
1. Момент, когда должен наступить следующий час, т.е. если значение секунд == 60, а значение минут == 59;
2. Момент, когда должен наступить следующий день (а час должен стать = 0), т.е. если значение секунд == 60, значение минут == 59, а значение часа == 23; 

In [ ]:
m_s = (
    data_preprocess
      .where(
          (f.col('minute') == '59') 
          & (f.col('second') == '60')
      )
      .count()
)

h_m_s = (
    data_preprocess
      .where(
          (f.col('hour') == '23') 
          & (f.col('minute') == '59') 
          & (f.col('second') == '60')
      )
      .count()
)

print(f'Количество строк в первой граничной ситуации: {m_s}, второй граничной ситуации: {h_m_s}')

Количество строк в первой граничной ситуации: 270, второй граничной ситуации: 2


Это наихудший случай. Если бы это значение было == 0, мы могли бы сделать сплит по символу ":" и взять лишь значение часов, т.к. важных данных мы бы не потеряли.

У решения этой проблемы 2 пути:

1. Починить колонку tr_datetime (вынести дни в отдельную колонку, привести время к нормальному виду, как это делается во втором решении) и работать именно с полем timestamp;
2. Сделать следующие действия:

    *   Сплит поля tr_datetime по символу пробела и отсечение значения дней;
    *   Сплит полученного поля по символу ':' и распределение часов, минут и секунд по разным полям;
    *   Использование два раза комбинации метода withColumn и функции when из модуля spark.sql.functions, для того, чтобы обработать обе граничные ситуации;

Т.к. из поля tr_datetime нам нужно лишь значение часов, в первом пути решения нет нужды.

Ожидаемый результат: если значение поля 'minute' == 59, а значение поля 'second' == 60, то значение поля 'hour' должно увеличиться на 1. Если значение поля 'hour' == 23, значение поля 'minute' == 59, а значение поля 'second' == 60, то значение поля 'hour' должно замениться на 0.

Проверим, получили ли мы ожидаемый результат:

In [ ]:
data_preprocess = df.withColumn('tr_datetime', f.split('tr_datetime', ' ')[1])

data_preprocess = (
    data_preprocess
      .withColumn('hour', f.split('tr_datetime', ':')[0])
      .withColumn('minute', f.split('tr_datetime', ':')[1])
      .withColumn('second', f.split('tr_datetime', ':')[2])
)

data_preprocess = (
    data_preprocess
      .withColumn(
          'hour', 
          f.when(
              (f.col('minute') == '59') 
              & (f.col('second') == '60'),
              (f.col('hour') + 1).cast(t.IntegerType())
          )
          .otherwise(f.col('hour').cast(t.IntegerType()))
      )
      .withColumn(
          'hour',
          f.when(
              (f.col('hour') == '24'),
              0
          )
          .otherwise(f.col('hour').cast(t.IntegerType()))
      )
)


(
    data_preprocess
      .select('tr_datetime', 'hour', 'minute', 'second')
      .where(
          (f.col('minute') == '59') 
          & (f.col('second') == '60')   
      ).show(1))

(
    data_preprocess
      .select('tr_datetime', 'hour', 'minute', 'second')
        .where(
            (f.split('tr_datetime', ':')[0] == '23') 
            & (f.split('tr_datetime', ':')[1] == '59')
            & (f.split('tr_datetime', ':')[2] == '60')
        ).show(1)
)

+-----------+----+------+------+
|tr_datetime|hour|minute|second|
+-----------+----+------+------+
|   14:59:60|  15|    59|    60|
+-----------+----+------+------+
only showing top 1 row

+-----------+----+------+------+
|tr_datetime|hour|minute|second|
+-----------+----+------+------+
|   23:59:60|   0|    59|    60|
+-----------+----+------+------+
only showing top 1 row



Как можно заметить, проблема решена и в поле "hour" у нас содержатся действительные значения часов. Выберем из датафрейма data_preprocess лишь те поля, которые нам нужны:

In [ ]:
req_cols = [col for col in data_preprocess.columns if col not in ['tr_datetime', 'minute', 'second']]
trans_data = data_preprocess[req_cols]

trans_data.show()

+-----------+--------+-------+---------+---------+----+
|customer_id|mcc_code|tr_type|   amount|direction|hour|
+-----------+--------+-------+---------+---------+----+
|   79780256|    4814|   1030|  3144.28|        D|  13|
|   79780256|    4814|   1030|  5614.79|        D|  10|
|   79780256|    6011|   2010|112295.79|        D|   9|
|   79780256|    6011|   2010| 67377.47|        D|   9|
|   79780256|    4814|   1030|  1122.96|        D|   8|
|   79780256|    4814|   1030|  2245.92|        D|   8|
|   79780256|    6011|   2010| 67377.47|        D|  11|
|   79780256|    6011|   2010| 22459.16|        D|  12|
|   79780256|    4814|   1030|  3368.87|        D|  15|
|   79780256|    4814|   1030|  2245.92|        D|  15|
|   79780256|    6011|   2010| 44918.32|        D|   6|
|   79780256|    6011|   2010| 89836.63|        D|  11|
|   79780256|    6011|   2010| 78607.05|        D|  10|
|   79780256|    6011|   2010|303198.63|        D|  12|
|   79780256|    6011|   2010| 67377.47|        

### 19. Сделать pivot, в котором строки это customer_id, столбцы часы, полученные на предыдущем этапе, в ячейках средние и стандартные отклонения по amount, значения должны быть разделены по направлениям

**Подсказка:** Можно сделать расчеты отдельно для каждого направления платежей, потом присоединить к заранее подготовленному списку уникальных customer_id. Так будет проще, наглядней и меньше вероятность сделать ошибку.

In [ ]:
d_dir = trans_data.where(f.col('direction') == 'D')
c_dir = trans_data.where(f.col('direction') == 'C')

In [ ]:
d_dir = (
    d_dir
      .groupBy('customer_id')
      .pivot('hour')
      .agg(
          f.mean('amount').alias('D_hour_avg'), 
          f.stddev('amount').alias('D_hour_std')
      )
)

c_dir = (
    c_dir
      .groupBy('customer_id')
      .pivot('hour')
      .agg(
          f.mean('amount').alias('C_hour_avg'), 
          f.stddev('amount').alias('C_hour_std')
      )
)

In [ ]:
trans_piv_hour = (
    dist_custs
      .join(
          d_dir, 
          'customer_id', 
          'left'
      )
      .join(
          c_dir, 
          'customer_id', 
          'left'
      )
)

trans_piv_hour.show()

+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------+------------------+--------

### 20. Соединить полученный DataFrame с pivot-ом по mcc кодам и по часам

**Примечание:** Суть тут в том, что мы по формируем набор данных, где для каждого customer_id мы имеем рассчитанные на основе транзакций признаки, такие как среднее арифметическое и стандартное отклонение сумм транзакций для каждого mcc кода, для каждого mcc с учетом направления транзакции, и для каждого часа в сутках без mcc кодов, но с учетом направления транзакции.

**Подсказка:** Список полей результирующего набора данных(… - другие аналогичные поля):

        ['customer_id',
         '742_mcc_avg',
         '742_mcc_std',
         '1711_mcc_avg',
         '1711_mcc_std',
         '1731_mcc_avg',
         '1731_mcc_std',
         ...
         ...
         ...
         '1010_c_type_avg',
         '1010_c_type_std',
         '1030_c_type_avg',
         '1030_c_type_std',
         '1100_c_type_avg',
         ...
         ...
         '1000_d_type_avg',
         '1000_d_type_std',
         '1010_d_type_avg',
         '1010_d_type_std',
         '1030_d_type_avg',
         ...
         ...
         '0_hour_c_avg',
         '0_hour_c_std',
         '1_hour_c_avg',
         '1_hour_c_std',
         '2_hour_c_avg',
         '2_hour_c_std',
         ...
         ...
         '23_hour_c_avg',
         '23_hour_c_std',
         '0_hour_d_avg',
         '0_hour_d_std',
         '1_hour_d_avg',
         '1_hour_d_std',
         ...
         ...
         '22_hour_d_avg',
         '22_hour_d_std',
         '23_hour_d_avg',
         '23_hour_d_std’]

---
Я правильно понимаю, что в примечание допущена ошибка (mcc коды не нужно было раскладывать по направлениям. Вместо этого нужно было разложить по направлениям типы транзакций)?

In [ ]:
features = (
    dist_custs
      .join(
          trans_piv_mcc_2, 
          'customer_id', 
          'left'
      )
      .join(
          trans_piv_type, 
          'customer_id', 
          'left'
      )
      .join(
          trans_piv_hour, 
          'customer_id', 
          'left'
      )
)

In [ ]:
features.show()

+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+----------------+-----------------+------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------------+-----------------+------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+------------+------------+-----------------+-----------------+------------+-----------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+---

Большое количество null значений имеет основание: не у каждого клиента были проводки в конкретный час с конкретным типом транзакции по конкретному mcc-коду.

Исходя из результата, у меня назрел такой вопрос:

Как я понял, данные фичи были собраны с целью скормить их модели. Не будет ли разумным все null значения превратить в 0. Null - отсутствие значения, но оно не даёт модели абсолютно никакой пользы. 0 в свою очередь даст модели понять, что в конкретный час с конкретным типом транзакции по конкретному mcc-коду у конкретного клиента проводок не было.

### 21. Какое кол-во столбцов получилось в итоговом DataFrame-е

In [ ]:
len(features.columns)

653

### 22. Сохранить результирующий датасет в csv-файл features.csv

In [ ]:
features.write.csv("features.csv")